In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
testData = pd.DataFrame()
for i in [1,2,4,5,6]:
    data = pd.read_json('../data/cleanQuestionList-{}.json'.format(str(i)))
    data['file'] = i
    testData = testData.append(data, ignore_index=True)
    
copyData = testData.copy()

# Clean dataframe

In [3]:
print('length of df: ', len(copyData))
copyData = copyData.replace(r'^\s*$', np.nan, regex=True)
copyData.describe()

length of df:  20036


,qNumber,answerNumber,file
count,20031.000000,19816.000000,20036.000000
mean,2454.159403,3.003482,3.707327
std,1462.581896,1.355855,1.679272
min,1.000000,1.000000,1.000000
25%,1252.500000,2.000000,2.000000
50%,2354.000000,3.000000,4.000000
75%,3545.500000,4.000000,5.000000
max,5684.000000,5.000000,6.000000


In [4]:
## missing data
for column in copyData.columns:
    print(column, len(copyData.loc[copyData[column].isnull()]))

questionDescription 3684
question 156
options 156
answer 5
qNumber 5
qOrigin 5
qTopic 156
answerNumber 220
answerDescription 220
file 0


In [5]:
# we need to remove those rows where question, options and answerNumber is null
copyData = copyData[copyData.question.notna() & 
                    copyData.options.notna() &
                    copyData.answerNumber.notna()
                   ]

In [6]:
## missing data
for column in copyData.columns:
    print(column, len(copyData.loc[copyData[column].isnull()]))

questionDescription 3655
question 0
options 0
answer 0
qNumber 0
qOrigin 0
qTopic 0
answerNumber 0
answerDescription 0
file 0


In [7]:
copyData.sample(4)

,questionDescription,question,options,answer,qNumber,qOrigin,qTopic,answerNumber,answerDescription,file
16276,A 38 year old male with known alcohol excess p...,What else would you do initially?,"{'1': 'IV antibiotics', '2': 'IV terlipressin ...",Answer & Comments Answer: 2- IV terlipressin ...,3532.0,ReviseMRCP,2012 September,2.0,Answer & Comments Answer: 2- IV terlipressin ...,5
18028,A 65 year old lady has epigastric pain for sev...,Which one of the following associations for He...,"{'1': 'Gastro oesophageal reflux disease', '2'...",Answer & Comments Answer: 2- Non ulcer formin...,1992.0,MRCPass,2010 September,2.0,Answer & Comments Answer: 2- Non ulcer formin...,6
7733,A 19-year-old student nurse was admitted after...,What diagnosis fits best with this clinical pi...,"{'1': 'Insulinoma', '2': 'Glucagonoma', '3': '...",Answer & Comments Answer: 4- Self-administrat...,674.0,PasTest 2009,Endocrinology,4.0,Answer & Comments Answer: 4- Self-administrat...,4
8350,A young woman with known systemic lupus erythe...,Which agent best fits her requirements?,"{'1': 'Mycophenolate mofetil', '2': 'Sirolimus...",Answer & Comments Answer: 1- Mycophenolate mo...,1291.0,PasTest 2009,Basic Science,1.0,Answer & Comments Answer: 1- Mycophenolate mo...,4


In [8]:
# remove duplicates

In [9]:
droppedDuplicate = copyData.drop_duplicates(subset=
    ['questionDescription', 'question']
)
len(droppedDuplicate)
# droppedDuplicate.questionDescription.value_counts()

14186

In [10]:
droppedDuplicate.qTopic.value_counts()

Endocrinology            1286
Basic Science            1241
Cardiology               1120
Neurology                1009
Rheumatology              952
Respiratory               952
Gastroenterology          925
Haematology               912
Nephrology                791
Infectious disease        692
Clinical pharmacology     664
Psychiatry                469
Dermatology               363
Statistics                261
Ophthalmology             201
2012 January              158
2011 January              150
2011 September            148
2010 September            139
2011 May                  139
General revision          136
Misc                      132
Tropical medicine         119
2010 May                  116
Emergency medicine        111
2009 January              109
2010 January              106
Pharmacology               98
2009 May                   96
2006 May                   83
2008 January               63
Emergency Medicine         54
2013 January               49
2008 Septe

In [14]:
def returnPastPaper(inputStr):
    if inputStr[0]=='2':
        return 'Past paper'
    else:
        return inputStr

droppedDuplicate.loc[:, 'qTopic'] = droppedDuplicate.loc[:, 'qTopic'].apply(lambda x: returnPastPaper(x))
droppedDuplicate.loc[droppedDuplicate['qTopic']=='Clinicalpharmacology', 'qTopic']='Clinical pharmacology'
droppedDuplicate.loc[droppedDuplicate['qTopic']=='Infectiousdisease', 'qTopic']='Infectious disease'
droppedDuplicate.loc[droppedDuplicate['qTopic']=='Tropicalmedicine', 'qTopic']='Tropical medicine'

In [15]:
for option in ['option1','option2','option3','option4','option5']:
    number = option[-1]
    droppedDuplicate.loc[:,option] = droppedDuplicate.loc[:, 'options'].apply(lambda x: x[number])

In [16]:
droppedDuplicate.sample(4)

,questionDescription,question,options,answer,qNumber,qOrigin,qTopic,answerNumber,answerDescription,file,option1,option2,option3,option4,option5
18383,A 26 year old lady presents with abdominal pai...,What is the likely diagnosis?,"{'1': 'Multiple myeloma', '2': 'Renal tubular ...",Answer & Comments Answer: 2- Renal tubular ac...,2163.0,MRCPass,Past paper,2.0,Answer & Comments Answer: 2- Renal tubular ac...,6,Multiple myeloma,Renal tubular acidosis type 1,Nephrotic syndrome,Homocystinuria,Porphyria
7835,A sexually active 19-year-old Nigerian man pre...,What is the most likely diagnosis?,"{'1': 'Herpes simplex virus', '2': 'Lymphogran...",Answer & Comments Answer: 1- Herpes simplex v...,776.0,PasTest 2009,Infectious disease,1.0,Answer & Comments Answer: 1- Herpes simplex v...,4,Herpes simplex virus,Lymphogranuloma venereum,Syphilis,Chancroid,Granuloma inguinale Part 1 ) – 2013 359
8997,A 72-year-old woman with metastatic carcinoma ...,Which of the following would be the most appro...,"{'1': 'Reduce her MST and add naproxen', '2': ...",Answer & Comments Answer: 4- Keep her MST sta...,1938.0,PasTest 2009,Clinical pharmacology,4.0,Answer & Comments Answer: 4- Keep her MST sta...,4,Reduce her MST and add naproxen,Keep her MST stable and stop her oral morphine,Stop her MST and start naproxen,"Keep her MST stable, reduce her oral morphine ...",Transfer her pain relief to fentanyl patches
4784,A 29-year-old woman is admitted for investigat...,Which of the following is the next most approp...,"{'1': '72 hour supervised fast', '2': 'CT abdo...",Answer & Comments Answer: 1- 72 hour supervis...,2309.0,OnExamination 2012,Endocrinology,1.0,Answer & Comments Answer: 1- 72 hour supervis...,2,72 hour supervised fast,CT abdomen,Pancreatic USS,Proinsulin measurement,Urine sulphonylurea assay


# explore data set

## need to look for duplicates!

In [17]:
exploreDataSet = droppedDuplicate.copy().reset_index().set_index('index')

In [18]:
exploreDataSet.describe()

,qNumber,answerNumber,file
count,14186.000000,14186.000000,14186.000000
mean,2042.849359,3.013887,3.105809
std,1396.291278,1.347912,1.487397
min,1.000000,1.000000,1.000000
25%,906.000000,2.000000,2.000000
50%,1825.000000,3.000000,4.000000
75%,3017.750000,4.000000,4.000000
max,5684.000000,5.000000,6.000000


In [19]:
exploreDataSet.head(10)

,questionDescription,question,options,answer,qNumber,qOrigin,qTopic,answerNumber,answerDescription,file,option1,option2,option3,option4,option5
index,,,,,,,,,,,,,,,
1,"A 45 year old man presents with acute, profuse...",Which one of the following is the most appropr...,"{'1': 'Metronidazole', '2': 'Ciprofloxacin', '...",Answer & Comments Answer: 2- Ciprofloxacin Th...,1.0,MRCPass,Gastroenterology,2.0,Answer & Comments Answer: 2- Ciprofloxacin Th...,1,Metronidazole,Ciprofloxacin,Vancomycin,Prednisolone,Cefuroxime
2,A 45 year old man has had long standing jaundi...,What is the likely diagnosis?,"{'1': 'Abnormal variant hepatitis C', '2': 'Su...",Answer & Comments Answer: 3- Hepatocellular c...,2.0,MRCPass,Gastroenterology,3.0,Answer & Comments Answer: 3- Hepatocellular c...,1,Abnormal variant hepatitis C,Superimposed hepatitis E infection,Hepatocellular carcinoma,Hepatitis A infection,Infectious mononucleosis
3,A 72 year old woman with longstanding hip oste...,What is the most likely cause?,"{'1': 'Non-steroidal anti-inflammatory drugs',...",Answer & Comments Answer: 5- Laxative abuse T...,3.0,MRCPass,Gastroenterology,5.0,Answer & Comments Answer: 5- Laxative abuse T...,1,Non-steroidal anti-inflammatory drugs,Crohn's disease,Diverticular disease,Whipple's disease,Laxative abuse
4,"A 40 year old man has symptoms of lethargy, jo...",What is the recommended management?,"{'1': 'Venesection', '2': 'Haemodialysis', '3'...",Answer & Comments Answer: 1- Venesection In h...,4.0,MRCPass,Gastroenterology,1.0,Answer & Comments Answer: 1- Venesection In h...,1,Venesection,Haemodialysis,Liver transplant,Phenoxybenzamine,Glucagon
5,A 50 year old patient has a 2 year history of ...,Which one of the following is a feature of VIP...,"{'1': 'Hypoglycaemia', '2': 'Hypokalaemia', '3...",Answer & Comments Answer: 2- Hypokalaemia VIP...,5.0,MRCPass,Gastroenterology,2.0,Answer & Comments Answer: 2- Hypokalaemia VIP...,1,Hypoglycaemia,Hypokalaemia,Induction of VIP release by somatostatin,Anaemia,Increased gastric acid seceretion
6,A 45 year old man has returned from holiday in...,What is the likely diagnosis?,"{'1': 'Campylobacter infection', '2': 'Coeliac...",Answer & Comments Answer: 1- Campylobacter in...,6.0,MRCPass,Gastroenterology,1.0,Answer & Comments Answer: 1- Campylobacter in...,1,Campylobacter infection,Coeliac disease,Tuberculosis,Ulcerative colitis,Gonococcal sepsis
7,A 22 year old man has recently returned from I...,Which of the following investigations would be...,"{'1': 'Ultrasound scan of abdomen', '2': 'Anti...",Answer & Comments Answer: 1- Ultrasound scan ...,7.0,MRCPass,Gastroenterology,1.0,Answer & Comments Answer: 1- Ultrasound scan ...,1,Ultrasound scan of abdomen,Anti endomysial antibody,Typhoid serology,"Stool for ova, cysts parasites",Colonosocopy
8,A 60 year old woman presents with diarrhoea. S...,What is the likely diagnosis?,"{'1': 'Radiation enteropathy', '2': 'Coeliac d...",Answer & Comments Answer: 2- Coeliac disease ...,8.0,MRCPass,Gastroenterology,2.0,Answer & Comments Answer: 2- Coeliac disease ...,1,Radiation enteropathy,Coeliac disease,Ischaemic colitis,Crohn's disease,Tropical Sprue
9,A 60 year old man has a 5 day history of abdom...,Which of the following is most likely to confi...,"{'1': 'CT scan of the abdomen', '2': 'Amoxycil...",Answer & Comments Answer: 3- Stool sample for...,9.0,MRCPass,Gastroenterology,3.0,Answer & Comments Answer: 3- Stool sample for...,1,CT scan of the abdomen,Amoxycillin and metronidazole,Stool sample for E coli 0157,Mesenteric angiography to exclude ischaemic co...,Surgical laparotomy


In [20]:
exploreDataSet.columns

Index(['questionDescription', 'question', 'options', 'answer', 'qNumber',
       'qOrigin', 'qTopic', 'answerNumber', 'answerDescription', 'file',
       'option1', 'option2', 'option3', 'option4', 'option5'],
      dtype='object')

In [21]:
slimDataSet = exploreDataSet[[
    'questionDescription', 'question', 'qTopic', 'option1', 'option2', 
    'option3', 'option4', 'option5', 'answerNumber']]

In [22]:
exploreDataSet.to_csv('../data/full-cleaned-pandas-ready-data-set.csv')
slimDataSet.to_csv('../data/slim-cleaned-pandas-ready-data-set.csv')